In [1]:
import json
import pandas as pd
import numpy as np
from dateutil.parser import parse
with open('Input_Game_Events.json') as f:
    org=json.load(f)
alist=[[[i['eventId'],i['studentId'],'DH' if i['playsetType']=='H' else 'DS',np.sum([(parse(k['roundEndTime'])-parse(k['roundStartTime'])).days*24*60+(parse(k['roundEndTime'])-parse(k['roundStartTime'])).seconds/60 for k in i['data']]),np.sum(len(j['responseText'])for j in i['data']), np.sum(1 if k['targetText'][0] in k['responseText'] else 0 for k in i['data'])] for i in tmp_dict] for tmp_dict in org.values()]
col_list=['EventId','StudentId','EventType','PlayMinutes','NoOfResponses','NoOfSuccessfulResponse']
df_stud_event=pd.DataFrame(alist[0],columns=col_list)
df_stud_event
df_stud_event.to_csv('Output_Games_Events_Discounts.txt',index=False)

In [20]:
import pandas as pd
import numpy as np
def calc_discount(row):
    if row['EventType']=='DH' and row['PlaysetType']=='H':
        return row['DiscountMinutes']+row['PlayMinutes']
    if row['EventType']=='DH' and row['PlaysetType']=='S':
        return row['DiscountMinutes']-row['PlayMinutes']
    if row['EventType']=='DS' and row['PlaysetType']=='H':
        return row['DiscountMinutes']-row['PlayMinutes']
    if row['EventType']=='DS' and row['PlaysetType']=='S':
        return row['DiscountMinutes']+row['PlayMinutes']
    
df_stud=pd.read_csv('Input_Start_Student_Discounts.txt')
df_event=pd.read_csv('Output_Games_Events_discounts.txt')
df_stud_evnt=df_stud.join(df_event.set_index('StudentId'),on='StudentId',how='inner')[['StudentId','PlaysetNumber','PlaysetType','DiscountMinutes','EventType','PlayMinutes']]
df_stud_evnt['DiscountMinutes']=df_stud_evnt.apply(calc_discount,axis=1)
df_final=df_stud_evnt.groupby(['StudentId','PlaysetNumber','PlaysetType'],as_index=False).DiscountMinutes.agg('sum')
df_min_max=df_final.agg({'DiscountMinutes':['max','min']})
df_final.to_csv('Output_End_Students_Discounts.txt',index=False)

DiscountMinutes   -3.0
Name: min, dtype: float64

In [32]:
df_final[(df_final['DiscountMinutes']==df_final['DiscountMinutes'].max()) | (df_final['DiscountMinutes']==df_final['DiscountMinutes'].min())][['StudentId']]

,StudentId
4,AS000200
6,MS000100
